Laboratorio 2 - Ramiro Sanes (368397) y Joaquín Guerra (307854)

# Reinforcement Learning Tarea 2 - Programación Dinámica (Grid World)

## Objetivos
- Entender y aplicar los conceptos de Programación Dinámica en Aprendizaje por Refuerzo.
- Implementar los algoritmos de Evaluación de Política (Policy Evaluation), Mejora de Política (Policy Improvement) e Iteración de Valor (Value Iteration).
- Obtener y analizar políticas óptimas en un ambiente de Grid World.

## A entregar
- Implementación del método de Iteración de Política.
- Implementación del método de Iteración de Valor.
- Comparación de los resultados obtenidos y discusión sobre convergencia y estabilidad.

## Descripción del ambiente a usar

Trabajaremos con el ambiente **Grid World** descrito en el capítulo 4 del libro de Sutton y Barto. Se trata de un entorno en forma de cuadrícula donde el agente puede moverse en cuatro direcciones:
- **RIGHT = 0** (derecha)
- **UP = 1** (arriba)
- **LEFT = 2** (izquierda)
- **DOWN = 3** (abajo)

El objetivo del agente es alcanzar uno de los dos estados terminales ubicados en la esquina superior izquierda y la esquina inferior derecha de la cuadrícula. Cada movimiento tiene un costo de -1, incentivando así la búsqueda de la ruta más corta hacia un estado terminal. Si el agente intenta moverse fuera de los límites de la cuadrícula, permanecerá en su posición actual.

Un ejemplo de un Grid World 4x4:
```
T  o  o  o
o  x  o  o
o  o  o  o
o  o  o  T
```
- "T" representa un estado terminal.
- "x" representa la posición actual del agente.
- "o" representa los estados transitables.

La dinámica de la recompensa es simple: el agente recibe un **recompensa de -1** en cada paso hasta alcanzar un estado terminal.

> Quizás se necesita:
> https://anaconda.org/conda-forge/gymnasium-box2d
> https://anaconda.org/conda-forge/gymnasium-other
> https://ffmpeg.org/download.html (en mac: `brew install ffmpeg`)

In [ ]:
import gymnasium as gym
from gymnasium.wrappers import RecordEpisodeStatistics, RecordVideo
import numpy as np
import GridWorldEnv

from Utils import print_state_values, print_state_action_values, print_policy_grids

In [ ]:
# Algunas constantes

HUMAN_RENDER = False # False si queremos guardar el episodio en un video, True si queremos verlo en tiempo real (no disponible en Google Colab u otros entornos sin interfaz gráfica)

RECORD_VIDEO = False # True si queremos guardar el episodio en un video, quizas se necesita: https://anaconda.org/conda-forge/gymnasium-other
RECORD_EVERY = 1 # Cada cuántos episodios guardamos un video

GRID_SIZE = 17 # Tamaño del grid
GAMMA = 1 # Factor de descuento

## Gymnasium Wrapper

En Gymnasium, los [wrappers](https://gymnasium.farama.org/api/wrappers/) son herramientas que permiten modificar el comportamiento de los entornos sin alterar su código base. Estos *wrappers* se utilizan para extender funcionalidades, como la normalización de observaciones, limitación de acciones o registro de estadísticas y videos durante el entrenamiento de agentes.

Para este ejercicio, se utilizarán dos *wrappers* de Gymnasium:

**`RecordVideo`**: Este *wrapper* permite grabar videos de las ejecuciones del agente en el entorno. Es especialmente útil para visualizar el comportamiento del agente y evaluar su desempeño. Se puede configurar para grabar todos los episodios o solo episodios específicos, según una función de activación definida por el usuario.

Documentación oficial de Gymnasium sobre [grabación de agentes](https://gymnasium.farama.org/introduction/record_agent/) y la [API de wrappers](https://gymnasium.farama.org/api/wrappers/). 

In [ ]:
def get_environment(human_render=HUMAN_RENDER, record_video=RECORD_VIDEO, record_every=RECORD_EVERY, grid_size=GRID_SIZE):
    assert not (human_render and record_video), "No se puede renderizar en tiempo real y guardar un video a la vez"
    
    render_mode = "human" if human_render else "rgb_array"
    
    # Initialise the environment
    env = gym.make("gymnasium_env/GridWorld-v0", render_mode=render_mode, size=grid_size)

    if record_video:
        env = RecordVideo(env, video_folder="./videos", name_prefix="gridworld",
                    episode_trigger=lambda x: x % record_every == 0)
    
    return env

Probamos el entorno y sus wrappers con un agente aleatorio para visualizar el comportamiento del entorno y la grabación de videos.

In [ ]:
# export IMAGEIO_FFMPEG_EXE
#import os
#os.environ["IMAGEIO_FFMPEG_EXE"] = "/opt/homebrew/bin/ffmpeg"

In [ ]:
env = get_environment()

# We play 10 episodes, each one with a random policy
for episode_num in range(10):
    obs, info = env.reset()

    episode_over = False
    while not episode_over:
        action = env.action_space.sample()  # random policy
        obs, reward, terminated, truncated, info = env.step(action)
        episode_over = terminated or truncated

env.close()

## Representación de la Política $\pi$

Para implementar los algoritmos de programación dinámica, necesitamos definir cómo representaremos la política $\pi(s, a)$. 

### Estructura de la Política
La política se almacenará en un **diccionario** de la forma:

```python
pi = {
    ((x, y), a): probabilidad
}
```

Donde:
- $(x, y)$ representa el estado en la grilla.
- $a$ representa una de las cuatro acciones posibles.
- `probabilidad` es la probabilidad de tomar la acción $a$ en el estado $(x, y)$.

Función auxiliar para saber si estamos en un estado final

In [ ]:
def is_done(state):
    return state in [(0, 0), (GRID_SIZE - 1, GRID_SIZE - 1)]

### Ejemplo: Política Uniforme
Una política uniforme en la que todas las acciones tienen la misma probabilidad ($ 25\% $) se define como:

In [ ]:
pi_rand = {}
for x in range(GRID_SIZE):
    for y in range(GRID_SIZE):
        for a in range(4):  # Cuatro acciones posibles
            if is_done((x, y)): # Si no estamos en un estado terminal
                continue
            pi_rand[((x, y), a)] = 1 / 4  # Probabilidad uniforme

print(pi_rand)

{((0, 1), 0): 0.25, ((0, 1), 1): 0.25, ((0, 1), 2): 0.25, ((0, 1), 3): 0.25, ((0, 2), 0): 0.25, ((0, 2), 1): 0.25, ((0, 2), 2): 0.25, ((0, 2), 3): 0.25, ((0, 3), 0): 0.25, ((0, 3), 1): 0.25, ((0, 3), 2): 0.25, ((0, 3), 3): 0.25, ((0, 4), 0): 0.25, ((0, 4), 1): 0.25, ((0, 4), 2): 0.25, ((0, 4), 3): 0.25, ((0, 5), 0): 0.25, ((0, 5), 1): 0.25, ((0, 5), 2): 0.25, ((0, 5), 3): 0.25, ((0, 6), 0): 0.25, ((0, 6), 1): 0.25, ((0, 6), 2): 0.25, ((0, 6), 3): 0.25, ((0, 7), 0): 0.25, ((0, 7), 1): 0.25, ((0, 7), 2): 0.25, ((0, 7), 3): 0.25, ((0, 8), 0): 0.25, ((0, 8), 1): 0.25, ((0, 8), 2): 0.25, ((0, 8), 3): 0.25, ((0, 9), 0): 0.25, ((0, 9), 1): 0.25, ((0, 9), 2): 0.25, ((0, 9), 3): 0.25, ((0, 10), 0): 0.25, ((0, 10), 1): 0.25, ((0, 10), 2): 0.25, ((0, 10), 3): 0.25, ((0, 11), 0): 0.25, ((0, 11), 1): 0.25, ((0, 11), 2): 0.25, ((0, 11), 3): 0.25, ((0, 12), 0): 0.25, ((0, 12), 1): 0.25, ((0, 12), 2): 0.25, ((0, 12), 3): 0.25, ((0, 13), 0): 0.25, ((0, 13), 1): 0.25, ((0, 13), 2): 0.25, ((0, 13), 3): 0

### Ejemplo: Política Determinista (derecha y abajo)
Si queremos que el agente siempre elija moverse a la derecha (acción `0`) a excepción de la ultima columna que se mueve hacia abajo (accion `3`), podemos definir la política de la siguiente manera:

In [ ]:
pi_der_aba = {}
for x in range(GRID_SIZE):
    for y in range(GRID_SIZE):
        if is_done((x, y)): # Si no estamos en un estado terminal
            continue
        if x == GRID_SIZE - 1:  # Si estamos en la última columna, solo podemos movernos hacia abajo
            pi_der_aba[((x, y), 3)] = 1
        else:  # En otro caso, podemos movernos hacia la derecha
            pi_der_aba[((x, y), 0)] = 1.0
print(pi_der_aba)

{((0, 1), 0): 1.0, ((0, 2), 0): 1.0, ((0, 3), 0): 1.0, ((0, 4), 0): 1.0, ((0, 5), 0): 1.0, ((0, 6), 0): 1.0, ((0, 7), 0): 1.0, ((0, 8), 0): 1.0, ((0, 9), 0): 1.0, ((0, 10), 0): 1.0, ((0, 11), 0): 1.0, ((0, 12), 0): 1.0, ((0, 13), 0): 1.0, ((0, 14), 0): 1.0, ((0, 15), 0): 1.0, ((0, 16), 0): 1.0, ((1, 0), 0): 1.0, ((1, 1), 0): 1.0, ((1, 2), 0): 1.0, ((1, 3), 0): 1.0, ((1, 4), 0): 1.0, ((1, 5), 0): 1.0, ((1, 6), 0): 1.0, ((1, 7), 0): 1.0, ((1, 8), 0): 1.0, ((1, 9), 0): 1.0, ((1, 10), 0): 1.0, ((1, 11), 0): 1.0, ((1, 12), 0): 1.0, ((1, 13), 0): 1.0, ((1, 14), 0): 1.0, ((1, 15), 0): 1.0, ((1, 16), 0): 1.0, ((2, 0), 0): 1.0, ((2, 1), 0): 1.0, ((2, 2), 0): 1.0, ((2, 3), 0): 1.0, ((2, 4), 0): 1.0, ((2, 5), 0): 1.0, ((2, 6), 0): 1.0, ((2, 7), 0): 1.0, ((2, 8), 0): 1.0, ((2, 9), 0): 1.0, ((2, 10), 0): 1.0, ((2, 11), 0): 1.0, ((2, 12), 0): 1.0, ((2, 13), 0): 1.0, ((2, 14), 0): 1.0, ((2, 15), 0): 1.0, ((2, 16), 0): 1.0, ((3, 0), 0): 1.0, ((3, 1), 0): 1.0, ((3, 2), 0): 1.0, ((3, 3), 0): 1.0, ((3, 4

### Ejemplo: Política Determinista (siempre abajo)
Si queremos que el agente siempre elija moverse hacia abajo (acción `3`), podemos definir la política de la siguiente manera:

In [ ]:
pi_abajo = {}
for x in range(GRID_SIZE):
    for y in range(GRID_SIZE):
        if is_done((x, y)): # Si no estamos en un estado terminal
                continue
        pi_abajo[((x, y), 3)] = 1

## Elección de una acción dado un estado y una política
Para elegir una acción dado un estado y una política, necesitamos muestrear una acción de acuerdo a la distribución de probabilidad definida en la política. Para esto, podemos utilizar la función `np.random.choice` de NumPy, que nos permite muestrear un elemento de un conjunto dado con una probabilidad dada.

Más información sobre `np.random.choice` en la [documentación oficial de NumPy](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html).

In [ ]:
def select_action(state, policy):
    """
    Dado un estado y una política, devuelve la acción a tomar.

    Parámetros:
    - state: Tupla (x, y) representando el estado actual.
    - policy: Diccionario {((x, y), a): probabilidad} con la política.

    Retorna:
    - La acción seleccionada según la política.
    
    Nota: Si la política es estocástica, se selecciona una acción al azar según las probabilidades.
    """
    actions = range(4)

    prob = [policy.get((state, a), 0) for a in actions]

    return np.random.choice(actions, p=prob)

select_action((2,2), pi_rand)

np.int64(0)

## Dinámica del Ambiente ($ p $) 

**La dinámica del ambiente $ p(s' | s, a) $**, que define las transiciones entre estados al tomar una acción. Esta información nos permite conocer, para cada estado y acción, cuál será la probabilidad de llegar al próximo estado y la recompensa recibida.

> Cuando creamos un ambiente con `gym.make()`, Gymnasium aplica un envoltorio (`OrderEnforcing`) que restringe el acceso directo a ciertos atributos internos. Para acceder a `p`, necesitamos usar `env.unwrapped`, que nos da acceso al objeto subyacente sin restricciones.

Ejemplo de acceso a la política desde el ambiente:


In [ ]:
p = env.unwrapped.p
print(p)

{((0, 0), 0): [(1.0, (0, 0), 0.0)], ((0, 0), 1): [(1.0, (0, 0), 0.0)], ((0, 0), 2): [(1.0, (0, 0), 0.0)], ((0, 0), 3): [(1.0, (0, 0), 0.0)], ((0, 1), 0): [(1.0, (1, 1), -1.0)], ((0, 1), 1): [(1.0, (0, 0), -1.0)], ((0, 1), 2): [(1.0, (0, 1), -1.0)], ((0, 1), 3): [(1.0, (0, 2), -1.0)], ((0, 2), 0): [(1.0, (1, 2), -1.0)], ((0, 2), 1): [(1.0, (0, 1), -1.0)], ((0, 2), 2): [(1.0, (0, 2), -1.0)], ((0, 2), 3): [(1.0, (0, 3), -1.0)], ((0, 3), 0): [(1.0, (1, 3), -1.0)], ((0, 3), 1): [(1.0, (0, 2), -1.0)], ((0, 3), 2): [(1.0, (0, 3), -1.0)], ((0, 3), 3): [(1.0, (0, 4), -1.0)], ((0, 4), 0): [(1.0, (1, 4), -1.0)], ((0, 4), 1): [(1.0, (0, 3), -1.0)], ((0, 4), 2): [(1.0, (0, 4), -1.0)], ((0, 4), 3): [(1.0, (0, 5), -1.0)], ((0, 5), 0): [(1.0, (1, 5), -1.0)], ((0, 5), 1): [(1.0, (0, 4), -1.0)], ((0, 5), 2): [(1.0, (0, 5), -1.0)], ((0, 5), 3): [(1.0, (0, 6), -1.0)], ((0, 6), 0): [(1.0, (1, 6), -1.0)], ((0, 6), 1): [(1.0, (0, 5), -1.0)], ((0, 6), 2): [(1.0, (0, 6), -1.0)], ((0, 6), 3): [(1.0, (0, 7), -1.

El diccionario tiene la siguiente forma:

```python
self.p = {
    ((x, y), a): [(probabilidad, (nuevo_x, nuevo_y), recompensa)]
}
```

Donde:
- `((x, y), a)`: Clave que representa el **estado actual** `(x, y)` y la **acción** `a` tomada.
- **Valor asociado**: Una lista con **una o más transiciones posibles** en el formato:
  - `probabilidad`: La probabilidad de que ocurra la transición (en este caso, siempre `1.0`, porque el ambiente es determinista).
  - `(nuevo_x, nuevo_y)`: El nuevo estado después de tomar la acción.
  - `recompensa`: El valor de recompensa por realizar la acción.

**Característica de la Política en este Caso**
- **Es determinista**, lo que significa que en cada estado hay una acción con probabilidad 1.
- **En los estados terminales, no importa la acción que se tome, simpre se permanece en el mismo estado.**

> ¿ Cómo podemos ver en el código estas propiedades ?

Para todas los valores del diccionario de la dinamica del ambiente, traemos el largo de la lista y luego cálculamos el máximo de estos largos.
En caso de que algun par estado acción tuviese más de un par estado/recompensa posibles, su lista debería tener largo 2, y por tanto el maximo largo debería ser 2.
Al obtener 1 como máximo, podemos asegurar que la dinámica del ambiente es determinista.

In [ ]:
max(len(i) for i in p.values())

1

Sabiendo que la dinámica del ambiente es determinista, solamente queda validar que en los estados terminales, para cualquier acción que se ejecute, el estado siguiente no se modifica:

In [ ]:
terminales = [(0, 0), (GRID_SIZE - 1, GRID_SIZE - 1)]
for s in terminales:
    for a in range(4):
        print(f"Estado Terminal: {s} y Acción: {a} -->> Estado Final: {p[s, a][0][1]}") # Probabilidad de ir a un estado terminal desde un estado terminal es 0

Estado Terminal: (0, 0) y Acción: 0 -->> Estado Final: (0, 0)
Estado Terminal: (0, 0) y Acción: 1 -->> Estado Final: (0, 0)
Estado Terminal: (0, 0) y Acción: 2 -->> Estado Final: (0, 0)
Estado Terminal: (0, 0) y Acción: 3 -->> Estado Final: (0, 0)
Estado Terminal: (16, 16) y Acción: 0 -->> Estado Final: (16, 16)
Estado Terminal: (16, 16) y Acción: 1 -->> Estado Final: (16, 16)
Estado Terminal: (16, 16) y Acción: 2 -->> Estado Final: (16, 16)
Estado Terminal: (16, 16) y Acción: 3 -->> Estado Final: (16, 16)


## Policy Iteration

La **Iteración de Política** (*Policy Iteration*) es un método fundamental en **Programación Dinámica** para encontrar la política óptima $\pi^*$ en un proceso de decisión de Markov (*MDP*) finito. Se basa en dos pasos clave que se repiten iterativamente hasta la convergencia:

1. **Evaluación de Política** (*Policy Evaluation*): Se calcula el valor de la política actual $\pi$, es decir, se obtiene la función de valor $V_\pi(s)$ resolviendo la ecuación de Bellman para todos los estados:
   $$
   V_\pi(s) = \sum_{a} \pi(a | s) \sum_{s', r} p(s', r | s, a) \left[ r + \gamma V_\pi(s') \right]
   $$
   donde $p(s', r | s, a)$ representa la dinámica del ambiente y $\gamma$ es el factor de descuento.

2. **Mejora de Política** (*Policy Improvement*): Se actualiza la política eligiendo en cada estado la acción que maximiza el valor esperado según la función de acción-valor $Q_\pi(s, a)$:
   $$
   \pi'(s) = \arg\max_{a} \sum_{s', r} p(s', r | s, a) \left[ r + \gamma V_\pi(s') \right]
   $$
   Si la nueva política $\pi'$ es diferente de la anterior, se repite el proceso con la nueva política. Si no cambia, hemos encontrado la política óptima $\pi^*$.

Proceso Iterativo:
1. Inicializar una política arbitraria $\pi$.
2. Aplicar **Policy Evaluation** para calcular $V_\pi$.
3. Aplicar **Policy Improvement** para obtener una nueva política $\pi'$.
4. Si $\pi' = \pi$, detenerse. De lo contrario, repetir desde el paso 2.

Este algoritmo garantiza la convergencia a la política óptima en un número finito de iteraciones en ambientes con estados y acciones finitas.

### Implementación de Policy evaluation

$$
\begin{aligned}
\textbf{Input:} \quad & \pi,\ \text{la política a evaluar.} \\
\textbf{Parámetro:} \quad & \theta > 0,\ \text{umbral pequeño que determina la precisión.} \\
\textbf{Inicializar:} \quad & V(s) \text{ arbitrariamente para todo } s \in S^+, \text{ excepto } V(\text{terminal}) = 0. \\
\\
\textbf{Repetir:} \quad & \Delta \gets 0 \\
& \text{Para cada } s \in S: \\
& \quad\quad v \gets V(s) \\
& \quad\quad V(s) \gets \sum_{a} \pi(a \mid s) \sum_{s',r} p(s',r \mid s,a)\,\Bigl[r + \gamma V(s')\Bigr] \\
& \quad\quad \Delta \gets \max\Bigl(\Delta,\, \bigl|v - V(s)\bigr|\Bigr) \\
\\
\textbf{Hasta que:} \quad & \Delta < \theta
\end{aligned}
$$



In [ ]:
def policy_evaluation(p, pi, gamma=GAMMA, theta=1e-5, size=GRID_SIZE):
    """
    Evalúa la política 'pi' en el entorno 'env' usando el método de evaluación iterativa.
    
    Parámetros:
        p: dinámica del entorno, diccionario que mapea (estado, acción) a una lista de (probabilidad, estado, recompensa).
        pi: politica a evaluar, diccionario que mapea (estado, acción) a su probabilidad en la política.
        gamma: factor de descuento.
        theta: umbral de convergencia.
    
    Retorna:
        V: función de valor de la política 'pi', diccionario que mapea estado (x, y) a su valor.
    """
    V = {}
    q = {}


    for y in range(size):
        for x in range(size):
            s = (x,y)
            V[s] = 0

    while True:
        delta = 0
        for y in range(size):
            for x in range(size):
                s = (x,y)
                v = V[s]
                pol = [pi.get((s, a), 0) for a in range(4)]

                for a in range(4):
                    q[(s,a)] = sum([prob * (r + gamma * V[s_siguiente]) for prob, s_siguiente, r in p[(s, a)]])

                V[s] = sum([pol[a] * q[(s,a)] for a in range(4)])
                delta = max(delta, abs(v - V[s]))

        if delta < theta:
            break

    return V

    
            


In [ ]:
[pi_abajo.get(((0,1), a), 0) for a in range(4)]

#print(pi_abajo[(0,1),1])

[0, 0, 0, 1]

In [ ]:
V_da_pi = policy_evaluation(p, pi_der_aba)
print_state_values(V_da_pi, GRID_SIZE)

    0.00   -31.00   -30.00   -29.00   -28.00   -27.00   -26.00   -25.00   -24.00   -23.00   -22.00   -21.00   -20.00   -19.00   -18.00   -17.00   -16.00
  -31.00   -30.00   -29.00   -28.00   -27.00   -26.00   -25.00   -24.00   -23.00   -22.00   -21.00   -20.00   -19.00   -18.00   -17.00   -16.00   -15.00
  -30.00   -29.00   -28.00   -27.00   -26.00   -25.00   -24.00   -23.00   -22.00   -21.00   -20.00   -19.00   -18.00   -17.00   -16.00   -15.00   -14.00
  -29.00   -28.00   -27.00   -26.00   -25.00   -24.00   -23.00   -22.00   -21.00   -20.00   -19.00   -18.00   -17.00   -16.00   -15.00   -14.00   -13.00
  -28.00   -27.00   -26.00   -25.00   -24.00   -23.00   -22.00   -21.00   -20.00   -19.00   -18.00   -17.00   -16.00   -15.00   -14.00   -13.00   -12.00
  -27.00   -26.00   -25.00   -24.00   -23.00   -22.00   -21.00   -20.00   -19.00   -18.00   -17.00   -16.00   -15.00   -14.00   -13.00   -12.00   -11.00
  -26.00   -25.00   -24.00   -23.00   -22.00   -21.00   -20.00   -19.00   -18.00  

In [ ]:
V_d_pi = policy_evaluation(p, pi_abajo, gamma=0.99)
print_state_values(V_d_pi, GRID_SIZE)

    0.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00   -14.85
 -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00   -13.99
 -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00   -13.13
 -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00   -12.25
 -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00   -11.36
 -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00   -10.47
 -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  

In [ ]:
V_pi_random = policy_evaluation(p, pi_rand, gamma=0.99)
print_state_values(V_pi_random, GRID_SIZE)

    0.00   -35.58   -55.32   -67.55   -75.69   -81.35   -85.40   -88.34   -90.50   -92.09   -93.26   -94.11   -94.73   -95.17   -95.46   -95.64   -95.73
  -35.58   -48.82   -61.02   -70.34   -77.19   -82.21   -85.91   -88.65   -90.68   -92.19   -93.30   -94.11   -94.70   -95.11   -95.39   -95.56   -95.64
  -55.32   -61.02   -68.03   -74.39   -79.60   -83.68   -86.82   -89.21   -91.02   -92.37   -93.37   -94.10   -94.63   -95.00   -95.24   -95.39   -95.46
  -67.55   -70.34   -74.39   -78.56   -82.30   -85.43   -87.95   -89.92   -91.44   -92.59   -93.44   -94.06   -94.50   -94.80   -95.00   -95.11   -95.17
  -75.69   -77.19   -79.60   -82.30   -84.91   -87.20   -89.13   -90.68   -91.89   -92.80   -93.48   -93.96   -94.29   -94.50   -94.63   -94.70   -94.73
  -81.35   -82.21   -83.68   -85.43   -87.20   -88.84   -90.24   -91.39   -92.30   -92.97   -93.45   -93.77   -93.96   -94.06   -94.10   -94.11   -94.11
  -85.40   -85.91   -86.82   -87.95   -89.13   -90.24   -91.22   -92.01   -92.62  

### Implementación de Policy Improvement

$$
\begin{aligned}
\textbf{Input:} \quad & V,\ \text{la función de valor evaluada}, \\
& p,\ \text{la dinamica del ambiente } \\
& \gamma,\ \text{el factor de descuento.} \\
\\
\textbf{Inicializar:} \quad & \text{Para cada } s \in S \text{ y } a \in A(s),\ new\_pi(s,a) \text{ se asigna arbitrariamente.} \\
\\
\textbf{Para cada } s \in S \text{ (excepto estados terminales):} \quad & \\
& \quad \text{Para cada acción } a \in A(s): \\
& \quad\quad Q(s,a) \gets \sum_{s',r} p(s',r \mid s,a) \Bigl[ r + \gamma\, V(s') \Bigr] \\
& \quad \text{Definir } best\_actions = \{ a \in A(s) \mid Q(s,a) = \max_{a' \in A(s)} Q(s,a') \} \\
& \quad \text{Para cada } a \in A(s): \\
& \quad\quad new\_pi(s,a) \gets 
\begin{cases}
\displaystyle \frac{1}{\lvert best\_actions \rvert}, & \text{si } a \in best\_actions, \\
0, & \text{si } a \notin best\_actions.
\end{cases} \\
\\
\textbf{Retornar:} \quad & new\_pi.
\end{aligned}
$$


In [ ]:
def calculate_Q(p, V, gamma=GAMMA, size=GRID_SIZE):
    """
    Calcula el valor Q(s, a) para cada estado y acción.
    
    Parámetros:
        p: dinámica del entorno, diccionario que mapea (estado, acción) a una lista de (probabilidad, estado, recompensa).
        V: función de valor, diccionario que mapea estado (x, y) a su valor.
        gamma: factor de descuento.
    
    Retorna:
        Q: valor Q(s, a).
    """
    Q = {}

    for y in range(size):
        for x in range(size):
            s = (x, y)
            for a in range(4):
                Q[(s, a)] = sum([prob * (r + gamma * V[next_s]) for (prob,next_s,r) in p[(s, a)]])

    return Q

In [ ]:
Q_pi_random = calculate_Q(p, V_pi_random)
print_state_action_values(Q_pi_random, GRID_SIZE)

Valores para la acción Derecha (acción 0):
    0.00   -56.32   -68.55   -76.69   -82.35   -86.40   -89.34   -91.50   -93.09   -94.26   -95.11   -95.73   -96.17   -96.46   -96.64   -96.73   -96.73
  -49.82   -62.02   -71.34   -78.19   -83.21   -86.91   -89.65   -91.68   -93.19   -94.30   -95.11   -95.70   -96.11   -96.39   -96.56   -96.64   -96.64
  -62.02   -69.03   -75.39   -80.60   -84.68   -87.82   -90.21   -92.02   -93.37   -94.37   -95.10   -95.63   -96.00   -96.24   -96.39   -96.46   -96.46
  -71.34   -75.39   -79.56   -83.30   -86.43   -88.95   -90.92   -92.44   -93.59   -94.44   -95.06   -95.50   -95.80   -96.00   -96.11   -96.17   -96.17
  -78.19   -80.60   -83.30   -85.91   -88.20   -90.13   -91.68   -92.89   -93.80   -94.48   -94.96   -95.29   -95.50   -95.63   -95.70   -95.73   -95.73
  -83.21   -84.68   -86.43   -88.20   -89.84   -91.24   -92.39   -93.30   -93.97   -94.45   -94.77   -94.96   -95.06   -95.10   -95.11   -95.11   -95.11
  -86.91   -87.82   -88.95   -90.13   -

In [ ]:
Q_da_pi = calculate_Q(p, V_da_pi)
print_state_action_values(Q_da_pi, GRID_SIZE)

Valores para la acción Derecha (acción 0):
    0.00   -31.00   -30.00   -29.00   -28.00   -27.00   -26.00   -25.00   -24.00   -23.00   -22.00   -21.00   -20.00   -19.00   -18.00   -17.00   -17.00
  -31.00   -30.00   -29.00   -28.00   -27.00   -26.00   -25.00   -24.00   -23.00   -22.00   -21.00   -20.00   -19.00   -18.00   -17.00   -16.00   -16.00
  -30.00   -29.00   -28.00   -27.00   -26.00   -25.00   -24.00   -23.00   -22.00   -21.00   -20.00   -19.00   -18.00   -17.00   -16.00   -15.00   -15.00
  -29.00   -28.00   -27.00   -26.00   -25.00   -24.00   -23.00   -22.00   -21.00   -20.00   -19.00   -18.00   -17.00   -16.00   -15.00   -14.00   -14.00
  -28.00   -27.00   -26.00   -25.00   -24.00   -23.00   -22.00   -21.00   -20.00   -19.00   -18.00   -17.00   -16.00   -15.00   -14.00   -13.00   -13.00
  -27.00   -26.00   -25.00   -24.00   -23.00   -22.00   -21.00   -20.00   -19.00   -18.00   -17.00   -16.00   -15.00   -14.00   -13.00   -12.00   -12.00
  -26.00   -25.00   -24.00   -23.00   -

In [ ]:
Q_d_pi = calculate_Q(p, V_d_pi, gamma=0.99)
print_state_action_values(Q_d_pi, GRID_SIZE)

Valores para la acción Derecha (acción 0):
    0.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00   -15.71   -15.71
 -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00   -14.85   -14.85
 -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00   -13.99   -13.99
 -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00   -13.13   -13.13
 -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00   -12.25   -12.25
 -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00   -11.36   -11.36
 -100.00  -100.00  -100.00  -100.00  -1

### Implementación de Policy Improvement

$$
\begin{aligned}
\textbf{Input:} \quad & V,\ \text{la función de valor evaluada}, \\
& p,\ \text{la dinamica del ambiente } \\
& \gamma,\ \text{el factor de descuento.} \\
\\
\textbf{Inicializar:} \quad & \text{Para cada } s \in S \text{ y } a \in A(s),\ new\_pi(s,a) \text{ se asigna arbitrariamente.} \\
\\
\textbf{Para cada } s \in S \text{ (excepto estados terminales):} \quad & \\
& \quad \text{Para cada acción } a \in A(s): \\
& \quad\quad Q(s,a) \gets \sum_{s',r} p(s',r \mid s,a) \Bigl[ r + \gamma\, V(s') \Bigr] \\
& \quad \text{Definir } best\_actions = \{ a \in A(s) \mid Q(s,a) = \max_{a' \in A(s)} Q(s,a') \} \\
& \quad \text{Para cada } a \in A(s): \\
& \quad\quad new\_pi(s,a) \gets 
\begin{cases}
\displaystyle \frac{1}{\lvert best\_actions \rvert}, & \text{si } a \in best\_actions, \\
0, & \text{si } a \notin best\_actions.
\end{cases} \\
\\
\textbf{Retornar:} \quad & new\_pi.
\end{aligned}
$$


In [ ]:
def improve_policy(p, V, gamma=GAMMA, size=GRID_SIZE):
    """
    Genera una política mejorada (greedy) a partir de la función de valor V.
    
    Parámetros:
        env: entorno que posee un atributo 'p' con la dinámica.
        V: diccionario que mapea estados a sus valores.
        gamma: factor de descuento.
    
    Retorna:
        new_pi: diccionario que representa la política mejorada, mapeando (estado, acción) a probabilidad.
    """

    new_pi = {}
    Q = calculate_Q(p, V, gamma)

    for y in range(size):
        for x in range(size):
            s = (x, y)
            if is_done(s):
                continue
            lista = np.array([Q[(s, a)] for a in range(4)])
            lista_indices = np.where(lista == np.max(lista))[0]
            for a in range(4):
                #new_pi[(s, a)] = 1 if a == mejor_a else 0
                new_pi[(s, a)] = 1/len(lista_indices) if a in lista_indices else 0
    
    return new_pi


In [ ]:
pi_mejorada = improve_policy(p, V_d_pi)

In [ ]:
pi_mejorada = improve_policy(p, V_d_pi)
V_mejorada = policy_evaluation(p, pi_mejorada)
print_state_values(V_mejorada, GRID_SIZE)

    0.00    -1.00   -80.33  -129.97  -161.75  -181.17  -191.17  -193.44  -189.09  -178.84  -163.22  -142.65  -117.44   -87.88   -54.28   -17.00   -16.00
   -1.00   -55.80  -106.00  -143.85  -170.10  -186.60  -194.88  -196.07  -190.98  -180.20  -164.18  -143.28  -117.78   -87.93   -53.95   -16.00   -15.00
  -87.46  -111.21  -140.04  -165.31  -184.21  -196.25  -201.69  -200.98  -194.56  -182.80  -166.03  -144.51  -118.48   -88.11   -53.58   -15.00   -14.00
 -146.18  -157.55  -173.63  -189.13  -201.18  -208.50  -210.66  -207.61  -199.47  -186.42  -168.63  -146.27  -119.50   -88.46   -53.26   -14.00   -13.00
 -189.52  -195.18  -203.80  -212.41  -218.88  -221.91  -220.83  -215.33  -205.31  -190.77  -171.79  -148.43  -120.79   -88.96   -53.01   -13.00   -12.00
 -223.19  -225.87  -229.97  -233.83  -236.01  -235.43  -231.42  -223.56  -211.65  -195.58  -175.32  -150.88  -122.29   -89.58   -52.81   -12.00   -11.00
 -250.18  -251.13  -252.37  -252.95  -251.89  -248.39  -241.85  -231.86  -218.15  

In [ ]:
def compare_policies(pi_1, pi_2, tol=1e-6, size=GRID_SIZE):
    """
    Compara dos políticas y retorna True si son iguales (considerando una tolerancia en los valores), o False en caso contrario.
    
    Parámetros:
        pi_1, pi_2: diccionarios que representan las políticas (mapean (estado, acción) a probabilidad).
        tol: tolerancia para comparar las probabilidades.
    
    Retorna:
        True si ambas políticas son iguales en todos los (estado, acción); False de lo contrario.
    """
    delta = 0
    for y in range(size):
        for x in range(size):
            s = (x, y)
            for a in range(4):
                if is_done(s):
                    continue
                # Comparamos las probabilidades de ambas políticas para el estado s y la acción a
                delta = max(delta, abs(pi_1.get((s, a), 0) - pi_2.get((s, a), 0)))

    

    return delta < tol

In [ ]:
def policy_iteration(p, pi_init, gamma=GAMMA, theta=1e-5, size=GRID_SIZE):
    """
    Realiza el algoritmo de iteración de política.
    
    Parámetros:
        p: dinámica del entorno, diccionario que mapea (estado, acción) a una lista de (probabilidad, estado, recompensa).
        gamma: factor de descuento.
        theta: umbral de convergencia.
    
    Retorna:
        pi: política óptima.
    """
    pi = pi_init
    iteration_counter = 0

    while True:
        V = policy_evaluation(p, pi, gamma, theta, size)
        new_pi = improve_policy(p, V, gamma=GAMMA, size=GRID_SIZE)
        if compare_policies(pi, new_pi):
            break
        else:
            pi = new_pi
            iteration_counter += 1
        
        

    return pi, iteration_counter

In [ ]:
pi_star = policy_iteration(p, pi_rand)[0]
print_policy_grids(pi_star, GRID_SIZE)

Política para la acción Derecha (acción 0):
  0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00
  0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.25   0.00
  0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.25   0.50   0.00
  0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.25   0.50   0.50   0.00
  0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.25   0.50   0.50   0.50   0.00
  0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.25   0.50   0.50   0.50   0.50   0.00
  0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.25   0.50   0.50   0.50   0.50   0.50   0.00
  0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.25   0.50   0.50   0.50   0.50   0.50   0.50   0.00
  0.

> 1) Explicar por qué esta política óptima tiene dicha distribución de probabilidad. 
> 2) Existen otras políticas óptimas posibles ?

1. Dado que cada par estado/accion tiene una misma recompensa en cada paso (-1) hasta llegar a alguno de los estados terminales, la acción optima a realizar desde cada estado, depende de la cantidad de pasos minimo desde estado siguiente hasta el estado terminal mas cercano.
Hay estados donde la política nos da la misma probabilidad para cualquiera de las acciones ya que la distancia desde cualquiera de los estados siguientes a alguno de los terminales es la misma.

2. Si existen otras, nosotros decidimos asignar a nuestra política pi(state,action) en los casos que la acción fuera la mejor 1/|mejores acciones|, si en lugar de esto hubieramos asignado, por ejemplo, pi(state,action) = 1 para solamente una de las mejores acciones, esta también hubiese sido una política óptima. Todas las políticas optimas deberían tener un mismo V(pi)


In [ ]:
V_pi_star = policy_evaluation(p, pi_star)
print_state_values(V_pi_star, GRID_SIZE)

    0.00    -1.00    -2.00    -3.00    -4.00    -5.00    -6.00    -7.00    -8.00    -9.00   -10.00   -11.00   -12.00   -13.00   -14.00   -15.00   -16.00
   -1.00    -2.00    -3.00    -4.00    -5.00    -6.00    -7.00    -8.00    -9.00   -10.00   -11.00   -12.00   -13.00   -14.00   -15.00   -16.00   -15.00
   -2.00    -3.00    -4.00    -5.00    -6.00    -7.00    -8.00    -9.00   -10.00   -11.00   -12.00   -13.00   -14.00   -15.00   -16.00   -15.00   -14.00
   -3.00    -4.00    -5.00    -6.00    -7.00    -8.00    -9.00   -10.00   -11.00   -12.00   -13.00   -14.00   -15.00   -16.00   -15.00   -14.00   -13.00
   -4.00    -5.00    -6.00    -7.00    -8.00    -9.00   -10.00   -11.00   -12.00   -13.00   -14.00   -15.00   -16.00   -15.00   -14.00   -13.00   -12.00
   -5.00    -6.00    -7.00    -8.00    -9.00   -10.00   -11.00   -12.00   -13.00   -14.00   -15.00   -16.00   -15.00   -14.00   -13.00   -12.00   -11.00
   -6.00    -7.00    -8.00    -9.00   -10.00   -11.00   -12.00   -13.00   -14.00  

In [ ]:
Q_pi_star = calculate_Q(p, V_pi_star)
print_state_action_values(Q_pi_star, GRID_SIZE)

Valores para la acción Derecha (acción 0):
    0.00    -3.00    -4.00    -5.00    -6.00    -7.00    -8.00    -9.00   -10.00   -11.00   -12.00   -13.00   -14.00   -15.00   -16.00   -17.00   -17.00
   -3.00    -4.00    -5.00    -6.00    -7.00    -8.00    -9.00   -10.00   -11.00   -12.00   -13.00   -14.00   -15.00   -16.00   -17.00   -16.00   -16.00
   -4.00    -5.00    -6.00    -7.00    -8.00    -9.00   -10.00   -11.00   -12.00   -13.00   -14.00   -15.00   -16.00   -17.00   -16.00   -15.00   -15.00
   -5.00    -6.00    -7.00    -8.00    -9.00   -10.00   -11.00   -12.00   -13.00   -14.00   -15.00   -16.00   -17.00   -16.00   -15.00   -14.00   -14.00
   -6.00    -7.00    -8.00    -9.00   -10.00   -11.00   -12.00   -13.00   -14.00   -15.00   -16.00   -17.00   -16.00   -15.00   -14.00   -13.00   -13.00
   -7.00    -8.00    -9.00   -10.00   -11.00   -12.00   -13.00   -14.00   -15.00   -16.00   -17.00   -16.00   -15.00   -14.00   -13.00   -12.00   -12.00
   -8.00    -9.00   -10.00   -11.00   -

In [ ]:
env = get_environment()

for episode_num in range(10):
    obs, info = env.reset()

    episode_over = False
    while not episode_over:
        state = (obs["pos"][0], obs["pos"][1])
        action = select_action(state, pi_star)
        obs, reward, terminated, truncated, info = env.step(action)
        episode_over = terminated or truncated

env.close()

## Value Iteration

Value Iteration es un algoritmo de programación dinámica empleado para estimar la función de valor óptima $ V^* $ y, a partir de ella, derivar la política óptima $ \pi^* $. Este método se basa en actualizar iterativamente los valores de cada estado utilizando la ecuación de Bellman, hasta que las actualizaciones sean menores que un pequeño umbral $ \theta $ que determina la precisión de la estimación.

El proceso general es el siguiente:

1. **Inicialización:**  
   Se asigna un valor arbitrario a $ V(s) $ para todos los estados $ s \in S^+ $, excepto en los estados terminales, donde se establece $ V(\text{terminal}) = 0 $.

2. **Actualización Iterativa:**  
   Para cada estado $ s \in S $, se actualiza su valor mediante la fórmula:
   $$
   V(s) \leftarrow \max_{a} \sum_{s', r} p(s', r \mid s, a) \Bigl[ r + \gamma V(s') \Bigr],
   $$
   donde $ p(s', r \mid s, a) $ es la probabilidad de transitar al estado $ s' $ y recibir la recompensa $ r $ al tomar la acción $ a $ en el estado $ s $.  
   La iteración continúa hasta que la diferencia máxima entre los valores antiguos y actualizados en todos los estados sea menor que $ \theta $.

3. **Extracción de la Política:**  
   Una vez convergido $ V $, se define la política óptima determinista $ \pi^* $ de la siguiente manera:
   $$
   \pi^*(s) = \operatorname*{argmax}_{a} \sum_{s', r} p(s', r \mid s, a) \Bigl[ r + \gamma V(s') \Bigr].
   $$

Este algoritmo es fundamental en el aprendizaje por refuerzo y en la resolución de procesos de decisión de Markov, ya que permite obtener de manera eficiente tanto la función de valor óptima como la política que maximiza el retorno esperado en cada estado.


In [ ]:
def value_iteration(p, gamma=GAMMA, theta=1e-5, size=GRID_SIZE):
    """
    Realiza el algoritmo de iteración de valor.
    
    Parámetros:
        p: dinámica del entorno, diccionario que mapea (estado, acción) a una lista de (probabilidad, estado, recompensa).
        gamma: factor de descuento.
        theta: umbral de convergencia.
    
    Retorna:
        pi: la pólitica óptima.
    """


    # Inicializamos la función de valor V y la política pi
    V = {(x,y):0 for x in range(size) for y in range(size)}
    V[(0,0)] = 0
    V[(size-1,size-1)] = 0
    V_prev = V.copy()
    dict_actions = {(x,y):[] for x in range(size) for y in range(size)}
    pi = {}
    iter_count = 0


    while True: 
        #print("entro")

        q = calculate_Q(p, V_prev, gamma, size)
        
        for x in range(size):
            for y in range(size):
                s = (x, y)
                if is_done(s):
                    continue
                
                #print(calculate_Q(p, V_prev, gamma, size))

                
                q_array = np.array([q[s,a] for a in range(4)])
                max_indexes = np.where(q_array == np.max(q_array))[0]

                #if np.max(q_array) >= V[s]:

                dict_actions[s] = max_indexes.tolist()
                V[s] = np.max(q_array)
        

        # Comprobamos la convergencia

        delta = max(abs(V[s] - V_prev[s]) for s in V.keys())
        if delta < theta:
            break
        else:
            V_prev = V.copy()  
            iter_count += 1 

    for state,actions in dict_actions.items():
        for action in actions:
                pi[(state, action)] = 1 / len(actions)

    return pi,iter_count
    


            
            

In [ ]:
pi_star2 = value_iteration(p)[0]
print_policy_grids(pi_star2, GRID_SIZE)


Política para la acción Derecha (acción 0):
  0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00
  0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.25   0.00
  0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.25   0.50   0.00
  0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.25   0.50   0.50   0.00
  0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.25   0.50   0.50   0.50   0.00
  0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.25   0.50   0.50   0.50   0.50   0.00
  0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.25   0.50   0.50   0.50   0.50   0.50   0.00
  0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.25   0.50   0.50   0.50   0.50   0.50   0.50   0.00
  0.

In [ ]:
env = get_environment()

for episode_num in range(10):
    obs, info = env.reset()

    episode_over = False
    while not episode_over:
        state = (obs["pos"][0], obs["pos"][1])
        action = select_action(state, pi_star2)
        obs, reward, terminated, truncated, info = env.step(action)
        episode_over = terminated or truncated

env.close()

## **Tareas**

**1. Comparación entre diferentes políticas iniciales**
- Prueba iniciar la **Iteración de Política** con diferentes políticas iniciales (aleatorias, deterministas, uniformes).
- ¿La política óptima cambia en función de la inicialización?
- ¿Cuántas iteraciones necesita cada caso para converger?
  
**2. [extra] Impacto de la estructura del Grid World**
- Modifica el tamaño de la grilla y analiza su impacto en el desempeño de los algoritmos.

**3. [extra] Evaluación en un entorno estocástico**
- Introduce aleatoriedad en la transición de estados (por ejemplo, que el agente no siempre se mueva exactamente en la dirección deseada). (usar "gymnasium_env/GridWorld_stochastic-v0")
- ¿Cómo cambia la política óptima en este caso?

__1. Comparación entre diferentes políticas iniciales__  

In [ ]:
print_state_values(V_d_pi, GRID_SIZE)
print("\n\n")



pol_abajo,iterations_abajo = policy_iteration(p, pi_abajo, 0.9, theta=1e-5, size=GRID_SIZE)

print(f"Cantidad de Iteraciones: {iterations_abajo} \n")
print_policy_grids(pol_abajo, GRID_SIZE)


    0.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00   -14.85
 -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00   -13.99
 -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00   -13.13
 -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00   -12.25
 -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00   -11.36
 -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00   -10.47
 -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  -100.00  

In [ ]:
print_state_values(V_da_pi, GRID_SIZE)
print("\n\n")

pol_der_aba,iterations_der_aba = policy_iteration(p, pi_der_aba, 0.9, theta=1e-5, size=GRID_SIZE)

print(f"Cantidad de Iteraciones: {iterations_der_aba} \n")
print_policy_grids(pol_der_aba, GRID_SIZE)



    0.00   -31.00   -30.00   -29.00   -28.00   -27.00   -26.00   -25.00   -24.00   -23.00   -22.00   -21.00   -20.00   -19.00   -18.00   -17.00   -16.00
  -31.00   -30.00   -29.00   -28.00   -27.00   -26.00   -25.00   -24.00   -23.00   -22.00   -21.00   -20.00   -19.00   -18.00   -17.00   -16.00   -15.00
  -30.00   -29.00   -28.00   -27.00   -26.00   -25.00   -24.00   -23.00   -22.00   -21.00   -20.00   -19.00   -18.00   -17.00   -16.00   -15.00   -14.00
  -29.00   -28.00   -27.00   -26.00   -25.00   -24.00   -23.00   -22.00   -21.00   -20.00   -19.00   -18.00   -17.00   -16.00   -15.00   -14.00   -13.00
  -28.00   -27.00   -26.00   -25.00   -24.00   -23.00   -22.00   -21.00   -20.00   -19.00   -18.00   -17.00   -16.00   -15.00   -14.00   -13.00   -12.00
  -27.00   -26.00   -25.00   -24.00   -23.00   -22.00   -21.00   -20.00   -19.00   -18.00   -17.00   -16.00   -15.00   -14.00   -13.00   -12.00   -11.00
  -26.00   -25.00   -24.00   -23.00   -22.00   -21.00   -20.00   -19.00   -18.00  

In [ ]:
print_state_values(V_pi_random, GRID_SIZE)
print("\n\n")

pol_rand,iterations_rand = policy_iteration(p, pi_rand, 0.9, theta=1e-5, size=GRID_SIZE)

print(f"Cantidad de Iteraciones: {iterations_rand} \n")
print_policy_grids(pol_rand, GRID_SIZE)



    0.00   -35.58   -55.32   -67.55   -75.69   -81.35   -85.40   -88.34   -90.50   -92.09   -93.26   -94.11   -94.73   -95.17   -95.46   -95.64   -95.73
  -35.58   -48.82   -61.02   -70.34   -77.19   -82.21   -85.91   -88.65   -90.68   -92.19   -93.30   -94.11   -94.70   -95.11   -95.39   -95.56   -95.64
  -55.32   -61.02   -68.03   -74.39   -79.60   -83.68   -86.82   -89.21   -91.02   -92.37   -93.37   -94.10   -94.63   -95.00   -95.24   -95.39   -95.46
  -67.55   -70.34   -74.39   -78.56   -82.30   -85.43   -87.95   -89.92   -91.44   -92.59   -93.44   -94.06   -94.50   -94.80   -95.00   -95.11   -95.17
  -75.69   -77.19   -79.60   -82.30   -84.91   -87.20   -89.13   -90.68   -91.89   -92.80   -93.48   -93.96   -94.29   -94.50   -94.63   -94.70   -94.73
  -81.35   -82.21   -83.68   -85.43   -87.20   -88.84   -90.24   -91.39   -92.30   -92.97   -93.45   -93.77   -93.96   -94.06   -94.10   -94.11   -94.11
  -85.40   -85.91   -86.82   -87.95   -89.13   -90.24   -91.22   -92.01   -92.62  

In [ ]:



if compare_policies(pol_der_aba,pol_rand, tol=1e-5, size=GRID_SIZE) and compare_policies(pol_der_aba,pol_abajo, tol=1e-5, size=GRID_SIZE) and compare_policies(pol_rand,pol_abajo, tol=1e-5, size=GRID_SIZE):
    print("Policy Iteration no cambia la política optima en función de la política inicial")
else:
    print("La política inicial afecta a la política óptima")

Policy Iteration no cambia la política optima en función de la política inicial


Las políticas "derecha-abajo" y "abajo" para el caso en el que GRID_SIZE = 4 necesitaron de 3 iteraciónes de policy iteration para converger mientras que la política uniforme necesitó de 2 iteraciones.

In [ ]:
v,iter_count = value_iteration(p, gamma=0.9, theta=1e-5, size=GRID_SIZE)
print(f"Cantidad de Iteraciones: {iter_count} \n")

Cantidad de Iteraciones: 16 



**2. [extra] Impacto de la estructura del Grid World**
- Al modificar el tamaño del grid, y aplicando policy iteration para las distintas políticas iniciales planteadas, observamos que la política uniforme, converge en 2 iteraciones para cualquier tamaño de grid (al menos hasta 20 que fue lo que simulamos). Esto puede ocurrir debido a que por la naturaleza de la política el V inicial ya contiene información que proviene de explorar cualquiera de los 4 estados siguientes.
En las otras políticas utilizadas, se necesitan más iteraciones para converger a medida que el GRID SIZE aumenta.

- Podemos ver que el algoritmo Value Iteration converge en una cantidad GRID SIZE -1 de iteraciones, debido a que es la distancia máxima del estado que está mas lejano al objetivo y el valor se propaga de a 1 celda por iteración.